In [1]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bi

In [4]:
!pip install flask-ngrok

In [12]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from transformers import pipeline, BertTokenizer, BertForTokenClassification
import torch

app = Flask(__name__)

# Load the BERT tokenizer and model for token classification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=18)
model.eval()

def anonymize_text(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        # Get the token-level predictions
        outputs = model(**inputs)

    # Get the predicted labels
    predicted_labels = torch.argmax(outputs.logits, dim=2).squeeze().tolist()
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].squeeze().tolist())

    anonymized_text = text

    # Replace identified entities with asterisks
    for token, label_id in zip(tokens, predicted_labels):
        label = model.config.id2label[label_id]
        if label != 'O':
            anonymized_text = anonymized_text.replace(token, '*' * len(token))

    return anonymized_text

@app.route('/anonymize', methods=['POST'])
def anonymize_email():
    if request.method == 'POST':
        data = request.get_json()
        email_text = data.get('email_text', '')

        anonymized_text = anonymize_text(email_text)

        response = {
            'anonymized_text': anonymized_text
        }
        return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
